In [1]:
!git clone https://github.com/YIKUAN8/Transformers-VQA.git
%cd Transformers-VQA/
!pip install -r requirements.txt

Cloning into 'Transformers-VQA'...
remote: Enumerating objects: 182, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 182 (delta 67), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (182/182), 1.94 MiB | 3.33 MiB/s, done.
Resolving deltas: 100% (67/67), done.
/content/Transformers-VQA
     |████████████████████████████████| 132 kB 4.3 MB/s 
     |████████████████████████████████| 8.6 MB 37.7 MB/s 
     |████████████████████████████████| 79 kB 4.5 MB/s 
     |████████████████████████████████| 138 kB 11.8 MB/s 
     |████████████████████████████████| 127 kB 39.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0

In [2]:
import pickle
from param import args
import pandas as pd

In [ ]:
#In param.py code, replace 'args = parser.parse_args()' with 'args = parser.parse_args([])' 

In [3]:
!wget https://convaisharables.blob.core.windows.net/uniter/pretrained/uniter-base.pt -P models/pretrained/

--2022-03-29 16:35:33--  https://convaisharables.blob.core.windows.net/uniter/pretrained/uniter-base.pt
Resolving convaisharables.blob.core.windows.net (convaisharables.blob.core.windows.net)... 13.77.184.64
Connecting to convaisharables.blob.core.windows.net (convaisharables.blob.core.windows.net)|13.77.184.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 273600756 (261M) [application/octet-stream]
Saving to: ‘models/pretrained/uniter-base.pt’

uniter-base.pt      100%[===================>] 260.93M  9.15MB/s    in 28s     

2022-03-29 16:36:02 (9.37 MB/s) - ‘models/pretrained/uniter-base.pt’ saved [273600756/273600756]



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
args.__dict__

{'batch_size': 32,
 'dropout': 0.1,
 'epochs': 2,
 'fast': False,
 'from_scratch': False,
 'load_pretrained': None,
 'load_trained': None,
 'lr': 0.0001,
 'max_seq_length': 20,
 'mce_loss': False,
 'model': 'lxmert',
 'multiGPU': False,
 'num_workers': 0,
 'optim': 'bert',
 'optimizer': 'bert',
 'output': 'models/trained/',
 'seed': 9595,
 'test': None,
 'tiny': False,
 'tqdm': True,
 'train': 'train,nominival',
 'valid': 'minival'}

In [6]:
args.batch_size = 18
args.epochs = 2
args.model = 'uniter' # use visualbert
args.load_pretrained = '/content/Transformers-VQA/models/pretrained/uniter-base.pt' #load pretrained visualbert model
args.max_seq_length = 128 #truncate or pad report lengths to 128 subwords

In [7]:
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import numpy as np
class PatientDataset(Dataset):
  def __init__(self, df, vf, model = 'visualbert'):
    self.visual_features = vf
    self.id_list = df.study_id.tolist()
    self.report_list = df.RadiologyReport.tolist()
    self.findings_list = df.columns[5:-2]
    self.target_list = df[self.findings_list].to_numpy().astype(np.float32)
    self.model = model

  def __len__(self):
    return len(self.id_list)

  def __getitem__(self, item):
    cxr_id = self.id_list[item]
    target = self.target_list[item]
    feats, boxes, img_w, img_h = self.visual_features[cxr_id]
    report = self.report_list[item]
    if self.model == 'uniter':
      boxes = self._uniterBoxes(boxes)
    if self.model == 'lxmert':
      boxes[:, (0, 2)] /= img_w
      boxes[:, (1, 3)] /= img_h
    #print(feats.shape, boxes.shape)
    return cxr_id, feats, boxes, report, target

  def _uniterBoxes(self, boxes):#uniter requires a 7-dimensiom beside the regular 4-d bbox
    new_boxes = np.zeros((boxes.shape[0],7),dtype='float32')
    new_boxes = np.zeros((boxes.shape[0],7),dtype='float32')
    new_boxes[:,1] = boxes[:,0]
    new_boxes[:,0] = boxes[:,1]
    new_boxes[:,3] = boxes[:,2]
    new_boxes[:,2] = boxes[:,3]
    new_boxes[:,4] = new_boxes[:,3]-new_boxes[:,1] #w
    new_boxes[:,5] = new_boxes[:,2]-new_boxes[:,0] #h
    new_boxes[:,6]=new_boxes[:,4]*new_boxes[:,5] #area
    return new_boxes  

In [8]:
!pip3 install pickle5
import pickle5 as pickle

     |████████████████████████████████| 256 kB 4.3 MB/s 


In [9]:
with open('/content/drive/MyDrive/researchLab/MIMIC_CXR_TESTING_DATASET_IMAGE.pkl', 'rb') as f:
        test_image = pickle.load(f)
with open('/content/drive/MyDrive/researchLab/MIMIC_CXR_TESTING_DATASET_TEXT.pkl', 'rb') as f:
        test_text = pickle.load(f)
with open('/content/drive/MyDrive/researchLab/MIMIC_CXR_TRAINING_DATASET_IMAGE.pkl', 'rb') as f:
        train_image = pickle.load(f)
with open('/content/drive/MyDrive/researchLab/MIMIC_CXR_TRAINING_DATASET_TEXT.pkl', 'rb') as f:
        train_text = pickle.load(f)

In [10]:
training = PatientDataset(df = train_text, vf = train_image, model = args.model)
testing = PatientDataset(df = test_text, vf = test_image, model = args.model)

In [11]:
assert set(list(test_image.keys())) == set(test_text.study_id.values), "Visual Features are inconsistent with openI dataset"

In [12]:
train_loader = DataLoader(training, batch_size=args.batch_size,shuffle=True, num_workers=0,drop_last=True, pin_memory=True)
test_loader = DataLoader(testing, batch_size=128,shuffle=False, num_workers=0,drop_last=False, pin_memory=True)

In [13]:
findings = list(train_text.columns[5:-2])
findings

['Atelectasis',
 'Cardiomegaly',
 'Consolidation',
 'Edema',
 'Enlarged Cardiomediastinum',
 'Fracture',
 'Lung Lesion',
 'Lung Opacity',
 'No Finding',
 'Pleural Effusion',
 'Pleural Other',
 'Pneumonia',
 'Pneumothorax']

In [14]:
from vqa_model import VQAModel
#init model
model = VQAModel(num_answers = len(findings), model = args.model)

100%|██████████| 404400730/404400730 [01:04<00:00, 6285817.86B/s]


In [15]:
#load pretrained weights
model.encoder.load(args.load_pretrained)

Load UNITER PreTrained Model from /content/Transformers-VQA/models/pretrained/uniter-base.pt

Weights in loaded but not in model:
cls.predictions.bias
cls.predictions.decoder.weight
cls.predictions.transform.LayerNorm.bias
cls.predictions.transform.LayerNorm.weight
cls.predictions.transform.dense.bias
cls.predictions.transform.dense.weight
feat_regress.bias
feat_regress.net.0.bias
feat_regress.net.0.weight
feat_regress.net.2.bias
feat_regress.net.2.weight
feat_regress.weight
itm_output.bias
itm_output.weight
region_classifier.net.0.bias
region_classifier.net.0.weight
region_classifier.net.2.bias
region_classifier.net.2.weight
region_classifier.net.3.bias
region_classifier.net.3.weight
uniter.img_embeddings.mask_embedding.weight

Weights in model but not in loaded:



In [16]:
#send to GPU
model = model.cuda()

In [17]:
import torch
loss = torch.nn.BCEWithLogitsLoss()

In [18]:
from src.optimization import BertAdam
optim = BertAdam(list(model.parameters()),lr=args.lr,warmup=0.1,t_total=len(train_loader)*args.epochs)
# t_total denotes total training steps
# batch_per_epoch = len(train_loader)
# t_total = int(batch_per_epoch * args.epochs)

In [19]:
# Evaluation function, we will report the AUC and accuray of each finding
import tensorflow as tf
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
accuracy = tf.keras.metrics.Accuracy()
AUC = tf.keras.metrics.AUC()

def eval(target, pred):
    b = np.zeros_like(pred)
    b[np.arange(len(pred)), pred.argmax(1)] = 1
    pred = b
    acc_list = []
    precision_list_tf = []
    recall_list_tf = []
    accuracy_list_tf = []
    AUC_list_tf = []
    precision_list_ker = []
    recall_list_ker= []
    accuracy_list_ker = []
    #print('Averaged Recall: '+str(recall_score(target, pred, average='weighted')))
    #print('Averaged Precision: '+str(precision_score(target, pred ,average='weighted')))
    #print('Averaged Accuracy: '+str(accuracy_score(target, pred)))
    for i, d in enumerate(findings[:-1]):
        precision.update_state(target[:,i], (pred[:,i]>=0.5))
        accuracy.update_state(target[:,i], (pred[:,i]>=0.5))
        recall.update_state(target[:,i], (pred[:,i]>=0.5))
        AUC.update_state(target[:,i], (pred[:,i]>=0.5))
        precision_list_ker.append(precision_score(target, pred ,average='weighted'))
        accuracy_list_ker.append(accuracy_score(target, pred))
        recall_list_ker.append(recall_score(target, pred, average='weighted'))
        precision_list_tf.append(precision.result().numpy())
        accuracy_list_tf.append(recall.result().numpy())
        recall_list_tf.append(accuracy.result().numpy())
        AUC_list_tf.append(AUC.result().numpy())
        acc = np.mean(target[:,i] == (pred[:,i]>=0.5))
        #print(i,d,acc)
        acc_list.append(acc)
    print('Averaged AUC : '+str(np.average(acc_list)))
    print('Averaged AUC tf : '+str(np.average(AUC_list_tf)))
    print('Averaged Recall tf: '+str(np.average(recall_list_tf)))
    print('Averaged Precision tf: '+str(np.average(precision_list_tf)))
    print('Averaged Accuracy tf: '+str(np.average(accuracy_list_tf)))
    print('Averaged Recall sklearn: '+str(np.average(recall_list_ker)))
    print('Averaged Precision sklearn: '+str(np.average(precision_list_ker)))
    print('Averaged Accuracy sklearn: '+str(np.average(accuracy_list_ker)))

In [20]:
sgmd = torch.nn.Sigmoid()

In [21]:
from param import args
args.__dict__

{'batch_size': 18,
 'dropout': 0.1,
 'epochs': 2,
 'fast': False,
 'from_scratch': False,
 'load_pretrained': '/content/Transformers-VQA/models/pretrained/uniter-base.pt',
 'load_trained': None,
 'lr': 0.0001,
 'max_seq_length': 128,
 'mce_loss': False,
 'model': 'uniter',
 'multiGPU': False,
 'num_workers': 0,
 'optim': 'bert',
 'optimizer': 'bert',
 'output': 'models/trained/',
 'seed': 9595,
 'test': None,
 'tiny': False,
 'tqdm': True,
 'train': 'train,nominival',
 'valid': 'minival'}

In [22]:
!pip install ipywidgets widgetsnbextension pandas-profiling

In [23]:
from tqdm.notebook import tqdm

iter_wrapper = (lambda x: tqdm(x, total=len(train_loader))) if args.tqdm else (lambda x: x)
best_valid = 0
for epoch in range(args.epochs):
  epoch_loss = 0
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(train_loader)):
    #print(cxr_id, feats, boxes, report, target)
    model.train()
    optim.zero_grad()
    feats, boxes, target = feats.cuda(), boxes.cuda(), target.cuda()
    logit = model(feats, boxes, report)
    running_loss = loss(logit, target)
    running_loss = running_loss * logit.size(1)
    epoch_loss += running_loss
    running_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
    optim.step()
  print("Epoch "+str(epoch)+": Training Loss: "+str(epoch_loss/len(train_loader)))
  print('Evaluation: ')
  model.eval()
  logit_list, target_list = [], []
  iter_wrapper = (lambda x: tqdm(x, total=len(test_loader)))
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(test_loader)):
    target_list.append(target)
    with torch.no_grad():
      feats, boxes = feats.cuda(), boxes.cuda()
      #print(feats, boxes)
      logit = model(feats, boxes, report)
      logit_list.append(sgmd(logit).cpu().numpy())
  eval(np.concatenate(target_list,axis = 0), np.concatenate(logit_list,axis = 0))

  0%|          | 0/833 [00:00<?, ?it/s]

/content/Transformers-VQA/src/optimization.py:142: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1050.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Epoch 0: Training Loss: tensor(4.0713, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/52 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

...

  0%|          | 0/52 [00:00<?, ?it/s]

Epoch 1: Training Loss: tensor(4.0537, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/52 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

Averaged AUC : 0.8444586594421032
Averaged AUC tf : 0.565643
Averaged Recall tf: 0.84835285
Averaged Precision tf: 0.33774832
Averaged Accuracy tf: 0.18524553
Averaged Recall sklearn: 0.21408128219805386
Averaged Precision sklearn: 0.07230559862318374
Averaged Accuracy sklearn: 0.33774834437086093


In [24]:
final_list = []

for index, item in enumerate(target_list):
    for index_1, item in enumerate(target_list[index]):
        print(logit_list[index][index_1])
        joined = target_list[index][index_1] +  logit_list[index][index_1]
        set_1 = [target_list[index][index_1].numpy(),logit_list[index][index_1]]
        final_list.append(set_1)
        #final_list.append(target_list[index][index_1]+logit_list[index][index_1])
# print(final_list)
df_final = pd.DataFrame(final_list, columns = ['actual', 'UNITER_prediction'])
df_final.to_pickle('UNITER_actual_prediction_testing.pkl')

Streaming output truncated to the last 5000 lines.
 0.01949393 0.22705479 0.3402352  0.24284442 0.00716161 0.08332953
 0.04152897]
[0.18334092 0.18911302 0.03884383 0.11635659 0.02663794 0.01794413
 0.01949393 0.22705473 0.34023526 0.24284448 0.00716161 0.08332954
 0.04152899]
[0.18334082 0.189113   0.03884384 0.11635661 0.02663789 0.01794412
 0.01949392 0.2270548  0.34023505 0.24284454 0.0071616  0.08332954
 0.04152896]
[0.18334088 0.18911302 0.03884384 0.11635661 0.0266379  0.01794411
 0.01949392 0.22705473 0.3402352  0.24284454 0.00716159 0.08332953
 0.04152897]
[0.18334085 0.18911295 0.03884381 0.11635666 0.02663791 0.01794413
 0.01949393 0.22705473 0.3402352  0.24284454 0.00716159 0.08332954
 0.04152899]
[0.18334088 0.18911299 0.03884383 0.11635664 0.02663793 0.01794412
 0.01949394 0.22705472 0.3402352  0.2428446  0.0071616  0.08332954
 0.04152899]
[0.18334092 0.18911304 0.03884385 0.11635659 0.02663794 0.01794414
 0.01949393 0.22705479 0.3402352  0.24284442 0.0071616  0.08332951
